# Python (Standalone)
## Unified Data batching & Agg function

### Installations

In [1]:
!pip install kafka

    100% |████████████████████████████████| 215kB 20.1MB/s ta 0:00:01


### Script

In [ ]:
from kafka import KafkaConsumer
import os
import glob
from datetime import datetime, timedelta
import time
import itertools
import json

# Select DF Factory
import cudf as pd
# import pandas as pd


# Basic configuration
metric_names = ['cpu_utilization', 'latency', 'packet_loss', 'throughput']
batch_len = 100
batch = list()

# Kafka configuration
topic = ''
servers = []
offset = 'earliest'

def handler(event):
    '''
        Processing function
    '''
    global batch
    global metric_names
    
    # Aggregate event jsons
    batch.append(event.body)
    
    # Did we aggregate enough events for aggregation?
    if len(batch) >= interval:
        
        # Create a DataFrame from the batch of event jsons
        df = cudf.read_json('\n'.join(batch), lines=True)
        df = df.reset_index(drop=True)
        
        # Perform aggregations
        df = df.groupby(['company']).\
                    agg({k: ['min', 'max', 'mean'] for k in metric_names})
        
        # Save to parquet
        filename = f'{time.time()}.parquet'
        filepath = os.path.join(sink, filename)
        new_index = [f'{e[0]}_{e[1]}' for e in list(df.columns)]
        df.columns = new_index
        df.to_parquet(filepath)
        
        # Reset batch
        batch = list()


# Kafka handling
consumer = KafkaConsumer(
     topic,
     bootstrap_servers=servers,
     auto_offset_reset='offset',
     value_deserializer=lambda x: x.decode('utf-8'))

for message in consumer:
    message = message.value
    handler(message)